In [240]:
from IPython.display import HTML

import jinja2 as jj
import numpy as np
import pandas as pd

In [235]:
def get_dataframe(name):
    data_url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
               f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    
    return pd.read_csv(data_url)

In [236]:
COL_COUNTRY = 'Country/Region'
COL_STATE = 'Province/State'
COUNTRY = 'Australia'

dft_confirm = get_dataframe('confirmed')
dft_confirm = dft_confirm[dft_confirm[COL_COUNTRY] == COUNTRY]
 
dft_demised = get_dataframe('deaths')
dft_demised = dft_demised[dft_demised[COL_COUNTRY] == COUNTRY]

dft_recover = get_dataframe('recovered')
dft_recover = dft_recover[dft_recover[COL_COUNTRY] == COUNTRY]

In [237]:
COL_TODAY = dft_confirm.columns[-1]
COL_1DAY = dft_confirm.columns[-1 - 1]
COL_5DAY = dft_confirm.columns[-1 - 5]
COL_50DAY = dft_confirm.columns[-1 - 50]

df_table = pd.DataFrame({'State': dft_confirm[COL_STATE], 
                         'Cases': dft_confirm[COL_TODAY], 'Deaths': dft_demised[COL_TODAY]})
df_table['Cases (5D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_5DAY])
df_table['Deaths (5D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_5DAY])
df_table['Cases (1D)'] = np.array(dft_confirm[COL_TODAY]) - np.array(dft_confirm[COL_1DAY])
df_table['Deaths (1D)'] = np.array(dft_demised[COL_TODAY]) - np.array(dft_demised[COL_1DAY])
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)

df_table = df_table.sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
df_table = df_table.reset_index()
df_table.index += 1

del df_table['index'] # del duplicate index
df_table.head(8)

,State,Cases,Deaths,Cases (5D),Deaths (5D),Cases (1D),Deaths (1D),Fatality Rate
1,New South Wales,1219,7,783,1,190,0,0.6
2,Victoria,520,3,291,3,54,3,0.6
3,Queensland,493,1,272,1,50,1,0.2
4,South Australia,235,0,168,0,65,0,0.0
5,Western Australia,231,2,141,1,56,1,0.9
6,Australian Capital Territory,53,0,44,0,14,0,0.0
7,Tasmania,47,0,31,0,11,0,0.0
8,Northern Territory,12,0,9,0,6,0,0.0


In [238]:
dt_cols = dft_confirm.columns[~dft_confirm.columns.isin([COL_STATE, COL_COUNTRY, 'Lat', 'Long'])]
dft_cases = dft_confirm.groupby(COL_STATE)[dt_cols].sum()
dft_cases_new = dft_cases.diff(axis=1).fillna(0).astype(int)

include_cols = ['Cases', 'Deaths', 'Cases (5D)', 'Deaths (5D)']

summary_nsw = df_table[df_table['State'].eq('New South Wales')][include_cols].sum().add_prefix('NSW ')
summary_vic = df_table[df_table['State'].eq('Victoria')][include_cols].sum().add_prefix('VIC ')
summary_qld = df_table[df_table['State'].eq('Queensland')][include_cols].sum().add_prefix('QLD ')

summary_time = {'updated': pd.to_datetime(COL_TODAY), 'since': pd.to_datetime(COL_5DAY)}
summary = {**summary_time, **df_table[include_cols].sum(), **summary_nsw, **summary_vic, **summary_qld}
summary

{'updated': Timestamp('2020-03-26 00:00:00'),
 'since': Timestamp('2020-03-21 00:00:00'),
 'Cases': 2810,
 'Deaths': 13,
 'Cases (5D)': 1739,
 'Deaths (5D)': 6,
 'NSW Cases': 1219,
 'NSW Deaths': 7,
 'NSW Cases (5D)': 783,
 'NSW Deaths (5D)': 1,
 'VIC Cases': 520,
 'VIC Deaths': 3,
 'VIC Cases (5D)': 291,
 'VIC Deaths (5D)': 3,
 'QLD Cases': 493,
 'QLD Deaths': 1,
 'QLD Cases (5D)': 272,
 'QLD Deaths (5D)': 1}

In [239]:
templateLoader = jj.FileSystemLoader(searchpath='./')
templateEnv = jj.Environment(loader=templateLoader)

template = templateEnv.get_template('templates/overview.html')
html_text = template.render(D=summary, table=df_table, newcases=dft_cases_new, np=np, pd=pd, enumerate=enumerate)

HTML(f'<div>{html_text}</div>')
